# Web Application Sidebar Refactoring Plan

This notebook outlines the CSS and component structure changes required to match the design specifications shown in the attached UI mockup. The goal is to modernize the left-hand control panel with a dark theme, specific typography, and improved interactive components.

## 1. Global Container Styles

First, we establish the foundational styles for the sidebar container to match the dark aesthetic.

In [ ]:
/* Global Container Styles */
aside {
    background-color: #000000; /* Pure Black Background */
    color: #FFFFFF;            /* Default White Text */
    font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
    padding: 20px;
    overflow-y: auto;
    display: flex;
    flex-direction: column;
}

## 2. Typography Rules

We need to enforce specific font weights for hierarchy. The section headers (like "George", "Depth & Effects") should be bold, while parameter labels should be medium weight.

In [ ]:
/* Typography Rules */

/* Section Headers (e.g., "George", "Depth & Effects") */
summary {
    font-family: 'Helvetica Neue', sans-serif;
    font-weight: 700; /* Bold */
    font-size: 14px;
    cursor: pointer;
}

/* Parameter Labels (e.g., "Trail Depth", "Spacing") */
.control-label {
    font-family: 'Helvetica Neue', sans-serif;
    font-weight: 500; /* Medium */
    font-size: 12px;
    color: #FFFFFF;
}

.control-value {
    font-weight: 400;
    font-size: 12px;
    opacity: 0.8;
}

## 3. Accordion / Section Header Styles

The design uses capsule-shaped headers for sections. These act as visual separators and collapsible triggers. When expanded, the entire panel turns white with black text.

In [ ]:
/* Section Header / Accordion Styles */
details {
    margin-bottom: 15px;
    border: 1px solid #333;
    border-radius: 4px;
    padding: 0;
    background-color: #000000;
    color: #ffffff;
    transition: all 0.3s ease;
}

/* Closed State */
summary {
    display: inline-block;
    background-color: #ffffff;
    color: #000000;
    border-radius: 20px;
    padding: 10px 15px;
    margin-bottom: 0;
    transition: all 0.2s ease;
}

/* Open/Active State */
details[open] {
    background-color: #ffffff;
    color: #000000;
    border-color: #ffffff;
}

details[open] > summary {
    background-color: #ffffff;
    color: #000000;
    border-radius: 20px;
    margin-bottom: 10px;
    display: block;
    width: auto;
}

## 4. Slider Control Layout

The sliders need a specific layout: Label on the left, Slider in the middle, Value on the right. We use Flexbox to achieve this alignment.

In [ ]:
/* Control Header Layout (Label + Value) */
.control-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-top: 15px;
    margin-bottom: 5px;
    padding: 0 10px;
}

details[open] .control-header {
    color: #000000;
}

/* Range Slider Styling */
input[type=range] {
    -webkit-appearance: none;
    width: 100%;
    background: transparent;
    margin: 5px 10px;
    padding: 0;
    box-sizing: border-box;
}

input[type=range]::-webkit-slider-runnable-track {
    background: #333;
    height: 2px;
    border-radius: 1px;
}

input[type=range]::-webkit-slider-thumb {
    -webkit-appearance: none;
    width: 12px;
    height: 12px;
    border-radius: 50%;
    background: #ffffff;
    cursor: pointer;
}

## 5. Label Updates

Based on the design requirements, specific text labels need to be updated in the HTML/React components.

**Changes:**
- Change `Copies` &rarr; `Trail Depth`
- Change `Copy Offset Scale` &rarr; `Trail Scale`
- Add new section header: `Direction` (replacing separate X/Y sliders with XY Pad)

In [ ]:
// Example: Control configuration updates
const controlsNumber = [
    {
        label: 'Trail Depth',  // Was 'Copies'
        min: 1,
        max: 30,
        path: 'parameters.copies.count'
    },
    {
        label: 'Trail Scale',  // Was 'Copy Offset Scale'
        min: -10,
        max: 20,
        path: 'parameters.copies.offset.scale'
    }
];

## 6. XY Coordinate Controller (2D Pad)

The design replaces separate X/Y sliders with a 2D control pad. This requires a custom component with a specific visual style: black background, white border, and crosshairs.

In [ ]:
/* XY Pad Container */
.xy-pad-container {
    width: calc(100% - 20px);
    aspect-ratio: 1 / 1; /* Square aspect ratio */
    border: 1px solid #666;
    position: relative;
    margin: 10px;
    cursor: crosshair;
    background: rgba(128, 128, 128, 0.1);
    margin-bottom: 10px;
}

details[open] .xy-pad-container {
    border-color: #ccc;
    background: rgba(0, 0, 0, 0.05);
}

/* Crosshair Lines */
.xy-pad-axis-x, .xy-pad-axis-y {
    position: absolute;
    background-color: currentColor;
    pointer-events: none;
    opacity: 0.3;
}

.xy-pad-axis-x {
    width: 100%;
    height: 1px;
    top: 50%;
    left: 0;
}

.xy-pad-axis-y {
    height: 100%;
    width: 1px;
    left: 50%;
    top: 0;
}

/* Draggable Handle */
.xy-pad-handle {
    width: 8px;
    height: 8px;
    background-color: currentColor;
    border-radius: 50%;
    position: absolute;
    transform: translate(-50%, -50%);
    pointer-events: none;
    box-shadow: 0 0 4px rgba(0, 0, 0, 0.2);
}

details[open] .xy-pad-handle {
    background-color: #000000;
}

### XY Pad Implementation (JavaScript)

In [ ]:
function renderXYPad(container) {
    // Create label header
    const label = document.createElement('div');
    label.className = 'control-header';
    label.innerHTML = '<span class="control-label">Direction</span>';
    container.appendChild(label);

    // Create XY pad container
    const padContainer = document.createElement('div');
    padContainer.className = 'xy-pad-container';
    
    // Add horizontal axis
    const xAxis = document.createElement('div');
    xAxis.className = 'xy-pad-axis-x';
    
    // Add vertical axis
    const yAxis = document.createElement('div');
    yAxis.className = 'xy-pad-axis-y';
    
    // Add draggable handle
    const handle = document.createElement('div');
    handle.className = 'xy-pad-handle';
    
    padContainer.appendChild(xAxis);
    padContainer.appendChild(yAxis);
    padContainer.appendChild(handle);
    container.appendChild(padContainer);
    
    // Handle drag logic
    let isDragging = false;
    padContainer.addEventListener('mousedown', () => { isDragging = true; });
    window.addEventListener('mouseup', () => { isDragging = false; });
    window.addEventListener('mousemove', (e) => {
        if (!isDragging) return;
        const rect = padContainer.getBoundingClientRect();
        const x = Math.max(0, Math.min(e.clientX - rect.left, rect.width));
        const y = Math.max(0, Math.min(e.clientY - rect.top, rect.height));
        handle.style.left = (x / rect.width * 100) + '%';
        handle.style.top = (y / rect.height * 100) + '%';
        // Update corresponding data values
    });
}

## 7. Bottom Navigation Buttons

The footer buttons ("About", "In Use", "Download") share a "capsule" style but are outlined rather than filled, reversing colors on hover.

In [ ]:
/* Bottom Button Styles */
#footer {
    display: flex;
    gap: 10px;
    flex-wrap: wrap;
    margin-top: auto;
    padding-top: 20px;
}

.capsule-btn {
    background-color: transparent;
    border: 1px solid #ffffff;
    color: #ffffff;
    border-radius: 20px;
    padding: 5px 15px;
    font-size: 12px;
    font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
    font-weight: 500;
    text-decoration: none;
    cursor: pointer;
    transition: all 0.2s ease;
    display: inline-block;
}

.capsule-btn:hover {
    background-color: #ffffff;
    color: #000000;
    border-color: #ffffff;
}

## 8. HTML Structure Example

Here's how the sidebar HTML should be structured:

In [ ]:
<aside>
    <div id="controls">
        <!-- George Section -->
        <details>
            <summary>George</summary>
            <div class="control-header">
                <span class="control-label">Width</span>
                <span class="control-value">1000</span>
            </div>
            <input type="range" min="200" max="2000" value="1000" />
        </details>

        <!-- Depth & Effects Section -->
        <details>
            <summary>Depth & Effects</summary>
            <div class="control-header">
                <span class="control-label">Trail Depth</span>
                <span class="control-value">15</span>
            </div>
            <input type="range" min="1" max="30" value="15" />
            
            <!-- XY Pad -->
            <div class="control-header">
                <span class="control-label">Direction</span>
            </div>
            <div class="xy-pad-container">
                <div class="xy-pad-axis-x"></div>
                <div class="xy-pad-axis-y"></div>
                <div class="xy-pad-handle"></div>
            </div>
        </details>

        <!-- Color Section -->
        <details>
            <summary>Color</summary>
            <!-- Color controls -->
        </details>
    </div>
    
    <!-- Footer Buttons -->
    <div id="footer">
        <a href="#" class="capsule-btn">About</a>
        <a href="#" class="capsule-btn">In Use</a>
        <a href="#" class="capsule-btn">Download</a>
    </div>
</aside>

## Summary of Changes

| Component | Change | Details |
|-----------|--------|---------|
| Background | Black (#000000) | Pure black for modern aesthetic |
| Typography | Bold (700) Headers, Medium (500) Labels | Helvetica Neue throughout |
| Accordion | White pill shape | Expands to white background with black text |
| Sliders | Flexbox layout | Label, slider, value in one row |
| XY Pad | New 2D control | Replaces separate X/Y sliders |
| Labels | Trail Depth, Trail Scale | Renamed from Copies and Copy Offset Scale |
| Buttons | White border, transparent background | Hover: invert colors (white bg, black text) |